# Duck typing

"If it walks like a duck, and quacks like a duck - it's a duck"

In [2]:
lst = [1,2,3]
dir(lst)

['__add__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__mul__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__rmul__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'append',
 'clear',
 'copy',
 'count',
 'extend',
 'index',
 'insert',
 'pop',
 'remove',
 'reverse',
 'sort']

To check whether an object is iterable, we check whether it implements the `__iter__` magic function

In [4]:
hasattr(lst, '__iter__')

True

# Magic functions

Standard methods are invoked by explicitly calling to them. However, a class can implement certain operations that are invoked by special syntax (such as arithmetic operations or subscripting and slicing) by defining methods with special names. These operations and their corresponding syntaxes are called **magic functions**, and they are a closed collection that can be found [here][1].

Magic functions are recognized by their leading and trailing underscores (e.g. \_\__init_\_\__()_), which are part of their names. In this chapter we will learn about some of the most common magic functions.

It should be noted that this is Python’s approach to a known programming concept called **operator overloading**, which allow classes to define their own behavior with respect to language operators.


[1]: https://docs.python.org/2/reference/datamodel.html#special-method-names "Magic functions documentation"

## Important magic functions

### \_\__str_\_\__(self)_

The \_\__str_\_\__()_ method returns the **string representation** of the instance, and this is by definition what the built-in function _print()_ shows when applied to an instance of the class.

### Rich comparison (\_\__cmp_\_\__(self, other)_)

The functionality of the comparison operators can be defined for class instances by implementing the magic functions \_\__lt_\_\__()_ ('<'), \_\__le_\_\__()_ ('<='), \_\__eq_\_\__()_ ('=='), \_\__ne_\_\__()_ ('!='), \_\__gt_\_\__()_ ('>') or \_\__ge_\_\__()_ ('>='). The input arguments are conventionally called _self_ and _other_ and the returned value is usually a Boolean.

Another cool feature of the \_\__cmp_\_\__(self, other)_ family is that it automatically enables the sorting of a list of instances, i.e. to apply the built-in function _sorted()_ on it.

> #### Note
> The operation '_self == other_' does **NOT** test whether _self_ and _other_ are the same **instance**, but rather if they have the same **value** as defined by the \_\__eq_\_\__(self, other)_ method. Testing if they are the same instance is performed via the _**is**_ operator. This is illustrated by the two snippets below.

In [1]:
a = [1]
b = [1]

print(a == b, a is b)

True False


In [2]:
a = [1]
b = a

print(a == b, a is b)

True True


### Emulating numeric types

Performing any arithmetic-like operation with class instances is possible with the corresponding magic function. The most common of them is \_\__add_\_\__()_ ('+') but every arithemtic operation has its magic function. The input arguments are conventionally called _self_ and _other_ and the returned value is a new instance of the class. The full documentation is available [here][1].

[1]: https://docs.python.org/2/reference/datamodel.html#emulating-numeric-types

### \_\__len_\_\__(self)_

The method \_\__len_\_\__()_ returns an integer representing the length of the instance.

### \_\__contains_\_\__(self)_

The method \_\__contains_\_\__()_ implements the behavior of the operator _in_.

### \_\__getitem_\_\__(self, key)_

The method \_\__getitem_\_\__(self, key)_ defines how the syntax _self[key]_ is implemented.

## The _Table_ example

### \_\__str_\_\__(self)_

The representation of a table is fairly intuitive. It is simply the tabular form of the table itself, unless it is too large to be displayed entirely, and then we may want to skip rows. 

### _len(self)_

Very intuitively, this method will return the number of records in the table.

### \_\__contains_\_\__(self)_

Since usually the first column of a table is associated with some unique key of its records, we will say that an item is _in_ the table if it is present in the first column of the table.

### Rich comparison (\_\__cmp_\_\__(self, other)_)

Since usually the first column of a table is associated with some unique key of its records, we will implement comparisons relating to the differences between the contents of the left-most column. A table A will be "greater than" a table B if all the keys of B are included in the keys of A.

> **NOTE:** This interpretation is very specific, and raises a lot of practical questions. However, it is used merely for illustration purposes, so don't take it too seriously.

### \_\__add_\_\__(self, other)_

This method will implement the union of the records of _self_ and _other_, regarding records with the same "key" as duplicates.

### \_\__getitem_\_\__(self, key)_

Understanding the left-most column as "keys", we would like the syntax _table[key]_ to return the record with the "key" _key_.

In [3]:
class Table:
    def __init__(self, *fields):
        self.fields = list(fields)
        self.n_fields = len(fields)
        self.records = []
        self.n_records = 0
    
    def __str__(self):
        template = "{:^25}" + (self.n_fields - 1) * "|{:^25}"
        if self.n_records < 10:
            records_str = "\n".join([template.format(*rec) for rec in self.records])
        else:
            records_str = "\n".join([template.format(*rec) for rec in self.records[:5]]) + "\n. . .\n"
            records_str += "\n".join([template.format(*rec) for rec in self.records[-5:]])

        header =  template.format(*self.fields) + "\n"
        horizontal_line = len(header) * "-" + "\n"
        return header + horizontal_line + records_str
    
    def __len__(self):
        return self.n_records
    
    def __contains__(self, item):
        return item in self.get_column(self.fields[0])
    
    def __eq__(self, other):
        if self.fields[0] != other.fields[0]:
            return False
        return set(self.get_column(self.fields[0])) == set(other.get_column(other.fields[0]))
    
    def __ne__(self, other):
        return not (self == other)
    
    def __lt__(self, other):
        if self.fields[0] != other.fields[0]:
            return False
        return set(self.get_column(self.fields[0])) < set(other.get_column(other.fields[0]))
    
    def __gt__(self, other):
        if self.fields[0] != other.fields[0]:
            return False
        return set(self.get_column(self.fields[0])) > set(other.get_column(other.fields[0]))

    def __le__(self, other):
        return not (self > other)
    
    def __ge__(self, other):
        return not (self < other)
    
    def __add__(self, other):
        if self.fields != other.fields:
            return None
        ret = Table(*self.fields)
        for rec in self.records:
            ret.add_record(rec)
        keys1 = self.get_column(self.fields[0])
        for rec in other.records:
            if rec[0] not in keys1:
                ret.add_record(rec)
        return ret      
        
    def __getitem__(self, key):
        return self.get_records(self.fields[0], key).records[0]
    
    def add_record(self, rec):
        self.records.append(rec)
        self.n_records += 1
        
    def remove_record(self, rec):
        self.records.remove(rec)
        self.n_records -= 1
        
    def get_column(self, field):
        ind = self.fields.index(field)
        return [rec[ind] for rec in self.records]        
                
    def get_records(self, field, value):
        if field in self.fields:
            ind = self.fields.index(field)
            ret = Table(*self.fields)
            for rec in self.records:
                if rec[ind] == value:
                    ret.add_record(rec)
            return ret
        else:
            print("{} is not a field in the table.".format(field))
            
    def get_fields(self, *fields):
        if all([(field in self.fields) for field in fields]):
            columns = [self.get_column(field) for field in fields]
            records = [list(rec) for rec in zip(*columns)]
            ret = Table(*fields)
            for rec in records:
                ret.add_record(rec)
            return ret
        else:
            print("{} is not a field in the table.".format(field))       

In [4]:
customers1 = Table('name', 'address', 'age')
customers1.add_record(['Russell Crowe', 'Dizengoff 4', 51])
customers1.add_record(['Nicolas Cage', 'Basel 7', 52])
customers1.add_record(['Diane Keaton', 'Basel 9', 52])
print(customers1)

          name           |         address         |           age           
------------------------------------------------------------------------------
      Russell Crowe      |       Dizengoff 4       |           51            
      Nicolas Cage       |         Basel 7         |           52            
      Diane Keaton       |         Basel 9         |           52            


In [5]:
customers2 = Table('name', 'address', 'age')
customers2.add_record(['Gwyneth Paltrow', 'Weizmann 8', 43])
customers2.add_record(['Al Pacino', 'Allenby 1', 63])
customers2.add_record(['Diane Keaton', 'Basel 9', 52])
print(customers2)

          name           |         address         |           age           
------------------------------------------------------------------------------
     Gwyneth Paltrow     |       Weizmann 8        |           43            
        Al Pacino        |        Allenby 1        |           63            
      Diane Keaton       |         Basel 9         |           52            


In [6]:
customers3 = Table('name', 'address', 'age')
customers3.add_record(['Russell Crowe', 'Dizengoff 4', 51])
customers3.add_record(['Nicolas Cage', 'Basel 7', 52])
customers3.add_record(['Gwyneth Paltrow', 'Weizmann 8', 43])
customers3.add_record(['Al Pacino', 'Allenby 1', 63])
customers3.add_record(['Diane Keaton', 'Basel 9', 52])
print(customers3)

          name           |         address         |           age           
------------------------------------------------------------------------------
      Russell Crowe      |       Dizengoff 4       |           51            
      Nicolas Cage       |         Basel 7         |           52            
     Gwyneth Paltrow     |       Weizmann 8        |           43            
        Al Pacino        |        Allenby 1        |           63            
      Diane Keaton       |         Basel 9         |           52            


In [7]:
print('Russell Crowe' in customers1)

True


In [8]:
print(customers1 < customers2)
print(customers1 < customers3)
print(customers3 >= customers2)

False
True
True


In [9]:
print(customers1 + customers2 == customers3)

True


In [10]:
print(customers1['Russell Crowe'])

['Russell Crowe', 'Dizengoff 4', 51]


# A moment of Zen

# What is a function ?

In [5]:
def func(x):
    return x*(x-1)/2

In [7]:
dir(func)

['__annotations__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [8]:
hasattr(func, '__call__')

True

# A function is anything that is callable

In [10]:
class PolynomialFunction:
    __slots__ = ['coefs']
    def __init__(self, *coefs):
        self.coefs = coefs
    def __call__(self, x):
        ret = 0
        for c in self.coefs:
            ret=ret*x+c
        return ret

In [13]:
quadratic=PolynomialFunction(1,2,1)
quadratic(1) # x^2+2x+1 == (x+1)^2

4

In [15]:
cubic=PolynomialFunction(1,3,3,1)
cubic(1) # x^3+3x^2+3x+1 == (x+1)^3

8

# Exercise
Rewrite the `PolynomialFunction` without using classes, with a function that returns a function